In [1]:
from IPython.display import HTML, Markdown, display

def markdown_print(string):
    display(Markdown(string))

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
def color_negative_red(value):
  """
  Colors elements in a dateframe
  green if positive and red if
  negative. Does not color NaN
  values.
  """

  if value < 0:
    color = 'red'
  elif value > 0:
    color = 'green'
  else:
    color = 'black'

  return 'color: %s' % color

In [3]:
#-------------------------------------------------------------------------------
# Name:        SV Comparison
# Purpose:     Debug merged SV instances
#
# Author:      kristjan.vilgo
#
# Created:     24.03.2019
# Copyright:   (c) kristjan.vilgo 2019
# Licence:     GPL V2
#-------------------------------------------------------------------------------
from RDF_parser import load_all_to_dataframe

from itertools import combinations

import pandas

from urlparse import urlparse

import os
import tempfile

from CGMEStools import *



# Pandas settings
pandas.set_option("display.max_rows", 50)
pandas.set_option("display.max_columns", 120)
pandas.set_option("display.width", 1000)
pandas.set_option('expand_frame_repr', True)
pandas.set_option('display.max_colwidth', -1) # Don't truncate cells



# Define here paths that you want to load. Load only SV files you want to compare, usually CGM SV files
paths = [r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_BALTICRSC-CE_003\20190514T0930Z_1D_BALTICRSC-CE_SV_003.zip",
         r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_CORESO-CE_002.zip",
         r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_HANS-CE_004.zip",         
         r"C:\IOPs\IOP150519\RSC_MERGE\20190515T0930Z_1D_TSCNET-CE_003_distributed_slack.zip",
         r"C:\Users\kristjan.vilgo\Downloads\20190304T0000Z_ENTSO-E_BD_001.zip",
         r"C:\IOPs\IOP150519\RSC_MERGE\IGM.zip"
]

data = load_all_to_dataframe(paths)




Loading 20190514T0930Z_1D_BALTICRSC-CE_SV_003.xml
Loading 20190514T0930Z_1D_BALTICRSC-CE-ELES_SSH_001.xml
Loading 20190514T0930Z_1D_BALTICRSC-CE-HOPS_SSH_001.xml
Loading 20190514T0930Z_1D_BALTICRSC-CE-NOSBIH_SSH_001.xml
Loading 20190304T0000Z_ENTSOE_EQBD_001.xml
Loading 20190304T0000Z_ENTSOE_TPBD_001.xml
Loading 20190515T0930Z_1D_HANS-hops_SSH_004.xml
Loading 20190515T0930Z_1D_HANS-nosbih_SSH_004.xml
Loading 20190515T0930Z_1D_HANS-CE_SV_004.xml
Loading 20190515T0930Z_1D_HANS-eles_SSH_004.xml
Loading 20190515T0930Z_1D_TSCNET-CE_SV_002.xml
Loading 20190515T0930Z_1D_TSCNET-ELES_SSH_002.xml
Loading 20190515T0930Z_1D_TSCNET-HOPS_SSH_002.xml
Loading 20190515T0930Z_1D_TSCNET-NOSBIH_SSH_001.xml
Loading 20190514T2230Z_HOPS_EQ_001.xml
Loading 20190515T0930Z_1D_ELES_SSH_001.xml
Loading 20190515T0930Z_1D_ELES_TP_001.xml
Loading 20190515T0930Z_1D_HOPS_SSH_001.xml
Loading 20190515T0930Z_1D_HOPS_TP_001.xml
Loading 20190515T0930Z_1D_NOSBIH_SSH_000.XML
Loading 20190515T0930Z_1D_NOSBIH_TP_000.XML
Loadin

In [4]:
filenames_list = []
for row, file_metadata in data.query("KEY=='Lable'").iterrows():
    file_meta_dict = get_metadata_from_filename(file_metadata["VALUE"])
    file_meta_dict["ID"] = file_metadata["INSTANCE_ID"]                                            
    filenames_list.append(file_meta_dict)

filename_data = pandas.DataFrame(filenames_list)
display(filename_data.sort_values("model_authority"))
#get_metadata_from_filename(data.query("KEY=='Lable'")["VALUE"].tolist()[7])

Warning - only 4 meta elements found, expecting 5, setting processtype to empty string
Warning - only 4 meta elements found, expecting 5, setting processtype to empty string
Warning - only 4 meta elements found, expecting 5, setting processtype to empty string
Warning - only 4 meta elements found, expecting 5, setting processtype to empty string
Warning - only 4 meta elements found, expecting 5, setting processtype to empty string


,ID,RSC,TSO,date_time,file_type,model_authority,process_type,profile,synchronous_area,version
0,09c51681-38df-4344-b712-34064bb5ba44,BALTICRSC,,20190514T0930Z,xml,BALTICRSC-CE,1D,SV,CE,003
1,0f3b5db8-a445-4f47-af76-a81906f3613e,BALTICRSC,ELES,20190514T0930Z,xml,BALTICRSC-CE-ELES,1D,SSH,CE,001
2,1fcfafda-3b14-4242-b7e6-ed166b86bdb6,BALTICRSC,HOPS,20190514T0930Z,xml,BALTICRSC-CE-HOPS,1D,SSH,CE,001
3,9c345dbe-842b-4da2-b4a5-96a7719d3b56,BALTICRSC,NOSBIH,20190514T0930Z,xml,BALTICRSC-CE-NOSBIH,1D,SSH,CE,001
23,2a80b480-5eb6-45e8-b15f-ac2dcf9b112b,,CGMEU,20190515T0930Z,xml,CGMEU,1D,SV,,003
16,0e2b93bb-8be8-4078-b0e3-521503ce0bce,,ELES,20190515T0930Z,xml,ELES,1D,TP,,001
15,b05d9199-9812-4f0c-88a5-395789cad57e,,ELES,20190515T0930Z,xml,ELES,1D,SSH,,001
24,b17569d2-dbb2-46b3-a70f-82f09cd5563f,,ELES,20190515T0930Z,xml,ELES,1D,SSH,,003
21,b132e996-490e-429b-aa4d-79351edf0c3f,,ELES,20190515T0930Z,xml,ELES,,EQ,,001
5,0b7890d9-72be-45fc-ae58-7f096dcda6cc,,ENTSOE,20190304T0000Z,xml,ENTSOE,,TPBD,,001


In [5]:
# Get all loaded FullModel headers
loaded_instances = data.type_tableview("FullModel")

# Configure the columns to be shown
column_list = [u'Model.modelingAuthoritySet', u'Model.description', u'Model.scenarioTime']

# Get unique profiles present
unique_profile_instances = loaded_instances["Model.profile"].unique().tolist()

profile_name_position = 4

# Display all by profile
for profile_instance in unique_profile_instances:

    # Print profile name    
    markdown_print("# " + profile_instance.split("/")[profile_name_position])
    
    # Display table of instaces of profile
    instances_data = loaded_instances[loaded_instances["Model.profile"] == profile_instance][column_list]
    #display(instances_data)
    
    # Display concrete classes statistics
    
    statistics_table = pandas.DataFrame()
    
    for instance_UUID, row in instances_data.sort_values("Model.modelingAuthoritySet").iterrows():
        
        column_name = filename_data.query("ID == '{}'".format(instance_UUID))["model_authority"].item()
        
        #row["Model.modelingAuthoritySet"] 
        statistics_table[column_name] = statistics_ConcreteClasses(data.query("INSTANCE_ID == '{}'".format(instance_UUID)))
        
    display(statistics_table)


# Topology

,ELES,HOPS,NOSBIH
ConnectivityNode,990,NaN,NaN
Terminal,923,1220.0,223.0
TopologicalNode,303,400.0,36.0
FullModel,1,1.0,1.0


# SteadyStateHypothesis

,BALTICRSC-CE-ELES,HANS-eles,ELES,HOPS,BALTICRSC-CE-HOPS,HANS-hops,NOSBIH,BALTICRSC-CE-NOSBIH,HANS-nosbih,TSCNET-HOPS,TSCNET-ELES,TSCNET-NOSBIH
Terminal,3385,4271,3385,1220.0,1220.0,1220.0,223.0,223.0,223.0,1220.0,3385,223.0
Disconnector,1181,1181,1181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1181,NaN
ConformLoad,178,178,178,NaN,353.0,NaN,34.0,34.0,34.0,NaN,178,34.0
SynchronousMachine,69,69,69,84.0,84.0,84.0,13.0,13.0,13.0,84.0,69,13.0
Breaker,52,52,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,NaN
HydroGeneratingUnit,51,51,51,40.0,40.0,40.0,3.0,3.0,3.0,40.0,51,3.0
RatioTapChanger,17,17,17,36.0,36.0,36.0,8.0,8.0,8.0,36.0,17,8.0
ThermalGeneratingUnit,17,17,17,23.0,23.0,23.0,NaN,NaN,NaN,23.0,17,NaN
EquivalentInjection,13,13,13,34.0,34.0,34.0,14.0,14.0,14.0,34.0,13,14.0
EnergySource,5,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN


# StateVariables

...

,ENTSOE
ConnectivityNode,549
TopologicalNode,549
FullModel,1


# EquipmentCore

,ELES,HOPS,NOSBIH
Terminal,3385,1220.0,223.0
Disconnector,1181,NaN,NaN
CurrentLimit,1116,1488.0,62.0
ConnectivityNode,990,NaN,NaN
OperationalLimitSet,636,744.0,98.0
ACLineSegment,240,325.0,54.0
VoltageLevel,180,209.0,36.0
ConformLoad,178,NaN,34.0
LoadResponseCharacteristic,178,NaN,36.0
PowerTransformerEnd,156,94.0,16.0


# EquipmentBoundary

,ENTSOE
ConnectivityNode,549
Line,549
BaseVoltage,64
EnergySchedulingType,12
FullModel,1
GeographicalRegion,1
SubGeographicalRegion,1


In [6]:
#loaded_instances #DEBUG

In [7]:
sv_instances = loaded_instances[loaded_instances["Model.profile"] == "http://entsoe.eu/CIM/StateVariables/4/1"]

for UUID, instance in sv_instances.iterrows():
    
    #display_name = str(instance["Model.description"])
    display_name = filename_data.query("ID == '{}'".format(UUID))["model_authority"].item()
    
    markdown_print("**Merge dependacies - " + display_name + "**")

    connections, identified_objects, graph = get_relations_from(data, UUID, notebook = True)
    display(graph.show(r"{}.html".format(UUID)))

**Merge dependacies - BALTICRSC-CE**

**Merge dependacies - CGMEU**

**Merge dependacies - HANS-CE**

**Merge dependacies - TSCNET-CE**

In [8]:
comparison_dict = {"statistics":{}, "data":{}, "report":{"Instances": loaded_instances}}
SV_UUID_list = data.query("VALUE == 'http://entsoe.eu/CIM/StateVariables/4/1'").ID.tolist()

EMF_namelist = []

for SV_UUID in SV_UUID_list:

    instance_data = data.query("INSTANCE_ID == '{}'".format(SV_UUID))
    #authority = instance_data.query("KEY == 'Model.modelingAuthoritySet'").VALUE.item()
    authority = filename_data.query("ID == '{}'".format(SV_UUID))["model_authority"].item()

    try:
        EMF_name = urlparse(authority).netloc

    except:
        print("No modelling authorityset found or invalid url -> {}, using SV UUID".format(authority))
        EMF_name = SV_UUID
        
    if EMF_name == "":
        print ("Not valid url -> " + authority + " , Using raw string")
        EMF_name = authority


    comparison_dict["statistics"]["{}".format(EMF_name)] = instance_data.types_dict()
    comparison_dict["data"]["{}".format(EMF_name)]       = instance_data

    EMF_namelist.append(EMF_name)


statistics = pandas.DataFrame(comparison_dict["statistics"])
comparison_dict["report"]["SvStatistics"] = statistics

statistics.fillna(0).T.style.applymap(color_negative_red).highlight_max(axis=0)

Not valid url -> BALTICRSC-CE , Using raw string
Not valid url -> HANS-CE , Using raw string
Not valid url -> TSCNET-CE , Using raw string
Not valid url -> CGMEU , Using raw string


,FullModel,SvInjection,SvPowerFlow,SvShuntCompensatorSections,SvStatus,SvTapStep,SvVoltage,TopologicalIsland
BALTICRSC-CE,1,0,2130,2,0,60,593,1
CGMEU,1,1,804,5,0,65,784,32
HANS-CE,1,0,864,5,0,65,782,271
TSCNET-CE,1,0,772,5,743,65,784,1


In [9]:

# Add here parameters to compere and at what index
settings = [dict(index = "SvTapStep.TapChanger",      merge_column = "SvTapStep.position"),
            dict(index = "SvPowerFlow.Terminal",      merge_column = "SvPowerFlow.p"),
            dict(index = "SvPowerFlow.Terminal",      merge_column = "SvPowerFlow.q"),
            dict(index = "SvVoltage.TopologicalNode", merge_column = "SvVoltage.v"),
            dict(index = "SvVoltage.TopologicalNode", merge_column = "SvVoltage.angle")]



# Create all comparison tables
for setting in settings:

    column_name = setting["merge_column"]
    type_name =   setting["merge_column"].split(".")[0]

    comparison_dict[column_name] = {}
    
    comparison_data = pandas.DataFrame()

    for SV_UUID in comparison_dict["data"].keys():

        rename_dict = {column_name:SV_UUID}

        data_view = comparison_dict["data"][SV_UUID].\
                    type_tableview(type_name).\
                    set_index(setting["index"])\
                    [[column_name]].\
                    rename(rename_dict, axis = "columns").\
                    apply(pandas.to_numeric, errors = "ignore")#, drop = False) # use .reset_index() before .set_index() to keep result UUID

        comparison_data = comparison_data.join(data_view, how = "outer")

    # Report all SV combinations
    combinations_iterator = combinations(comparison_dict["data"].keys(), 2) # Create all unique combnations of merged model providers

    for combination in combinations_iterator:

        diff_column_name = "{} - {}".format(combination[0], combination[1])
        comparison_data[diff_column_name] = comparison_data[combination[0]] - comparison_data[combination[1]]

    comparison_dict["report"][column_name] = comparison_data

    # Add statistics

    #columns = [u'<baltic-rsc.eu> - <rte-france.fr>', u'<baltic-rsc.eu> - <nordic-rsc.net>', u'<baltic-rsc.eu> - <tscnet.eu>', u'<baltic-rsc.eu> - <scc-rsci.com>']

    comparison_dict["report"][column_name + "_" + "statistics"] = comparison_dict["report"][column_name][EMF_namelist].describe()

    
    
# Save to excel    
#excel_writer = pandas.ExcelWriter(r"C:\IOPs\IOP150519\RSC_MERGE\SV_comparison.xlsx")

#for report in comparison_dict["report"].keys():
#    comparison_dict["report"][report].to_excel(excel_writer, sheet_name = report)

#excel_writer.save()


# Some info

#print("done")


#print(loaded_profiles[loaded_profiles["Model.profile"]=="http://entsoe.eu/CIM/StateVariables/4/1"])

#print("all data is avaialbel in 'comparison_dict['report']'")
#print(comparison_dict['report'].keys())



In [10]:
statisitcs_columns = ["count", "min", "max", "std"]

statistics_to_report = ['SvTapStep.position_statistics', 
                        'SvPowerFlow.p_statistics',
                        'SvPowerFlow.q_statistics',
                        'SvVoltage.v_statistics']

# Number of differences to show
show_top = 5

for report_name in statistics_to_report:
    markdown_print("# " + report_name)
    display(comparison_dict['report'][report_name].T[statisitcs_columns].round(2).style.applymap(color_negative_red).highlight_min(axis=0))

    diff_comparison = comparison_dict['report'][report_name.split("_")[0]].dropna()
    print ("Number of common points -> " + str(len(diff_comparison)))
    diff = pandas.DataFrame(diff_comparison.drop(columns = EMF_namelist).abs().sum(), columns = ["sum diff"]).round(2).sort_values("sum diff")
    display(diff)
    
    largest_diff = diff.tail(1).index.item()
    largest_diff_value = diff.tail(1)["sum diff"].item()
    
    if float(largest_diff_value) > 0:

        markdown_print("**Max positive diff**")
        show_columns = EMF_namelist + [largest_diff]
        sorted_diff = diff_comparison[show_columns].sort_values(largest_diff,kind= 'mergesort', ascending = False)
        display(sorted_diff.head(show_top))

        from_UUID = sorted_diff.index[0]
        connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        display(graph.show(r"{}.html".format(from_UUID)))

        markdown_print("**Max negative diff**")
        display(sorted_diff.tail(show_top))

        from_UUID = sorted_diff.index[-1]
        connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        display(graph.show(r"{}.html".format(from_UUID)))
        
    else:
        print("No differences")
        
    

# SvTapStep.position_statistics

,count,min,max,std
BALTICRSC-CE,60,-17,21,9.19
HANS-CE,65,-17,21,9.25
TSCNET-CE,65,-17,21,9.25
CGMEU,65,-17,21,9.25


Number of common points -> 60


,sum diff
CGMEU - BALTICRSC-CE,0.0
CGMEU - HANS-CE,0.0
CGMEU - TSCNET-CE,0.0
BALTICRSC-CE - HANS-CE,0.0
BALTICRSC-CE - TSCNET-CE,0.0
HANS-CE - TSCNET-CE,0.0


No differences


...

,sum diff
CGMEU - TSCNET-CE,28.78
BALTICRSC-CE - HANS-CE,42.97
CGMEU - HANS-CE,67.47
CGMEU - BALTICRSC-CE,68.18
BALTICRSC-CE - TSCNET-CE,80.96
HANS-CE - TSCNET-CE,82.27


**Max positive diff**

,BALTICRSC-CE,HANS-CE,TSCNET-CE,CGMEU,HANS-CE - TSCNET-CE
SvVoltage.TopologicalNode,,,,,
0f36f2c98d8d4e3d88e3abe68ffe05a4,396.9386,396.965,396.539,396.977,0.426
00a95f620fa6400b9443781141692f5e,397.6507,397.677,397.252,397.689,0.425
2416a679-626c-7a5d-ac39-ed056b2ba4ef,397.7422,397.769,397.345,397.780,0.424
09d0c878-951d-23cd-aeda-c7fd133221e8,110.8915,110.924,110.510,110.959,0.414
f14e3a40-f5f5-8471-a378-c80a15ae6e0c,111.2513,111.265,110.861,111.301,0.404


**Max negative diff**

,BALTICRSC-CE,HANS-CE,TSCNET-CE,CGMEU,HANS-CE - TSCNET-CE
SvVoltage.TopologicalNode,,,,,
59f5b6afd47a40ab9df63321cb698e29,408.5439,408.326,409.478,409.206,-1.152
9d25a1f9e5d14d47b6dcde99c4380b40,408.5402,408.325,409.478,409.205,-1.153
e268acaf-1e6c-4e33-ac80-e02668b94273,407.9077,407.695,408.848,408.575,-1.153
8403e6cd-e1ba-49a2-ab09-f65b90a396cd,405.5249,405.439,406.832,406.420,-1.393
b25d3280-2222-4db8-b2ba-1eeed9752723,405.1936,405.200,406.759,406.278,-1.559


In [11]:
voltages = pandas.DataFrame()

markdown_print("**Count of common voltages**")

for name in EMF_namelist:

    voltages[name] = comparison_dict['report'][report_name.split("_")[0]][name].dropna().value_counts()
    
voltages.dropna()

**Count of common voltages**

,BALTICRSC-CE,HANS-CE,TSCNET-CE,CGMEU
0.0,101,290.0,292.0,292.0


In [12]:
count_column_name = "Number_of_TopologicalNodes"
min_nodes_count = 4 # Number of nodes in topological Island for it to be reported 

markdown_print("**Topological Islands with greater than {} Topological Nodes**".format(min_nodes_count))


# Filter all topological nodes belonging to topological island
TopologicalIsland_TopologicalNodes = data.query("KEY == 'TopologicalIsland.TopologicalNodes'")

# Count number of topological nodes in an island
TopologicalIsland_TopologicalNodes_count = pandas.DataFrame(TopologicalIsland_TopologicalNodes["ID"].value_counts()).reset_index().rename(columns = {"index":"ID", "ID":count_column_name})

# Topological islands names
TopologicalIslands_report = data.query("KEY == 'IdentifiedObject.name'")

# Merge with count
TopologicalIslands_report = TopologicalIslands_report.merge(TopologicalIsland_TopologicalNodes_count, on="ID")

# Merge with insatnce metadata
#TopologicalIslands_report = TopologicalIslands_report.merge(sv_instances, left_on="INSTANCE_ID", right_index=True)
TopologicalIslands_report = TopologicalIslands_report.merge(filename_data, left_on="INSTANCE_ID", right_on = "ID")
#display(TopologicalIslands_report)

# Sort, filter and select columns for display
TopologicalIslands_report.sort_values(count_column_name, ascending = False)[['ID_x', 'VALUE', count_column_name, 'model_authority']]\
                        .query("{} > {}".format(count_column_name, min_nodes_count)).set_index("ID_x")
#[['ID', 'VALUE', count_column_name, 'Model.modelingAuthoritySet', 'Model.description']]\


**Topological Islands with greater than 4 Topological Nodes**

,VALUE,Number_of_TopologicalNodes,model_authority
ID_x,,,
fba45f7e-d9dd-4622-bf38-e2eb1e889e7a,TopoIsland 1,492,BALTICRSC-CE
e6c402c2-c7cc-4241-9937-af9032600221,N1,492,TSCNET-CE
f213bf73-a70a-55f6-be14-23b56b494f62,_f213bf73-a70a-55f6-be14-23b56b494f62,458,CGMEU
45fd2cc9-8f70-4bf6-9f15-fda1b9555512,Island_001,452,HANS-CE
4f9e1b08-f87e-5eb6-a370-87af25710b9a,_4f9e1b08-f87e-5eb6-a370-87af25710b9a,216,CGMEU


In [13]:
# Get diff of nodes between two islands

#data.query("(ID == 'e6c402c2-c7cc-4241-9937-af9032600221' or ID == 'f213bf73-a70a-55f6-be14-23b56b494f62') and KEY == 'TopologicalIsland.TopologicalNodes'").drop_duplicates(["VALUE"],keep = False).VALUE.tolist()


In [14]:
markdown_print("# Synchronous machine comparison")
display(statistics_GeneratingUnit_types(data).round(1)) 

# Synchronous machine comparison

,VALUE,TOTAL,%
HydroGeneratingUnit,94,166,56.6
ThermalGeneratingUnit,40,166,24.1
WindGeneratingUnit,21,166,12.7
GeneratingUnit,10,166,6.0
NuclearGeneratingUnit,1,166,0.6


In [15]:
# Get Synchronous Machines, Terminals, Generating Units and Regulating controls
Terminals               = data.type_tableview("Terminal")
SynchronousMachines     = data.type_tableview("SynchronousMachine")

# Merge with Terminals
SynchronousMachines = pandas.merge(SynchronousMachines.reset_index(),
                                   Terminals.reset_index(), 
                                   suffixes=('', '_Terminal'),
                                   how = "inner",
                                   left_on = "ID",
                                   right_on = 'Terminal.ConductingEquipment')

# Query for referenced objects
GeneratingUnits    = tableview_by_IDs(data, SynchronousMachines,"RotatingMachine.GeneratingUnit")
RegulatingControls = tableview_by_IDs(data, SynchronousMachines,"RegulatingCondEq.RegulatingControl")

# Add data from referenced objects
SynchronousMachines = pandas.merge(SynchronousMachines,
                                   GeneratingUnits,
                                   left_on = "RotatingMachine.GeneratingUnit",
                                   right_index = True,
                                   how= "left",
                                   suffixes=('', '_GeneratingUnit'))

SynchronousMachines = pandas.merge(SynchronousMachines,
                                   RegulatingControls,
                                   left_on = "RegulatingCondEq.RegulatingControl",
                                   right_index = True,
                                   how= "left",
                                   suffixes=('', '_RegulatingControl'))










In [16]:
SynchronousMachines

KEY,ID,Equipment.EquipmentContainer,Equipment.aggregate,IdentifiedObject.description,IdentifiedObject.name,RegulatingCondEq.RegulatingControl,RegulatingCondEq.controlEnabled,RotatingMachine.GeneratingUnit,RotatingMachine.p,RotatingMachine.q,RotatingMachine.ratedPowerFactor,RotatingMachine.ratedS,RotatingMachine.ratedU,SynchronousMachine.InitialReactiveCapabilityCurve,SynchronousMachine.earthing,SynchronousMachine.earthingStarPointR,SynchronousMachine.earthingStarPointX,SynchronousMachine.ikk,SynchronousMachine.maxQ,SynchronousMachine.minQ,SynchronousMachine.mu,SynchronousMachine.operatingMode,SynchronousMachine.qPercent,SynchronousMachine.r,SynchronousMachine.r0,SynchronousMachine.r2,SynchronousMachine.referencePriority,SynchronousMachine.satDirectSubtransX,SynchronousMachine.satDirectSyncX,SynchronousMachine.satDirectTransX,SynchronousMachine.shortCircuitRotorType,SynchronousMachine.type,SynchronousMachine.voltageRegulationRange,SynchronousMachine.x0,SynchronousMachine.x2,Type,ID_Terminal,ACDCTerminal.connected,ACDCTerminal.sequenceNumber,IdentifiedObject.description_Terminal,IdentifiedObject.name_Terminal,Terminal.ConductingEquipment,Terminal.ConnectivityNode,Terminal.TopologicalNode,Terminal.phases,Type_Terminal,Equipment.EquipmentContainer_GeneratingUnit,Equipment.aggregate_GeneratingUnit,GeneratingUnit.genControlSource,GeneratingUnit.initialP,GeneratingUnit.maxOperatingP,GeneratingUnit.minOperatingP,GeneratingUnit.nominalP,GeneratingUnit.normalPF,HydroGeneratingUnit.energyConversionCapability,IdentifiedObject.description_GeneratingUnit,IdentifiedObject.name_GeneratingUnit,Type_GeneratingUnit,WindGeneratingUnit.windGenUnitType,IdentifiedObject.name_RegulatingControl,RegulatingControl.Terminal,RegulatingControl.discrete,RegulatingControl.enabled,RegulatingControl.mode,RegulatingControl.targetDeadband,RegulatingControl.targetValue,RegulatingControl.targetValueUnitMultiplier,Type_RegulatingControl
0,008245da-d767-4ab5-8221-b447b44b0f28,60d5f4cb-6c8c-4ca7-9da9-543fb97b0ced,NaN,NaN,G1,d33bd8e5-ba8e-a7bd-4f1b-8a900755480c,false,0f7e5013-491e-bc9c-883e-c1e1228a3e26,-4,-0.3463693,0,48,110,NaN,NaN,NaN,NaN,NaN,24.8,-24.8,NaN,http://iec.ch/TC57/2013/CIM-schema-cim16#SynchronousMachineOperatingMode.generator,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,http://iec.ch/TC57/2013/CIM-schema-cim16#SynchronousMachineKind.generator,NaN,NaN,NaN,SynchronousMachine,018344db-d666-4bb4-8320-b546b54a0e29,true,1,NaN,T1,008245da-d767-4ab5-8221-b447b44b0f28,NaN,4c8c48e8-3d63-e78a-2406-c9b49d48dac0,NaN,Terminal,2bf441db-2393-4c9b-940e-5df6bd731b5c,NaN,NaN,0,48,0,NaN,1,NaN,NaN,G1,WindGeneratingUnit,http://iec.ch/TC57/2013/CIM-schema-cim16#WindGenUnitKind.onshore,G1,018344db-d666-4bb4-8320-b546b54a0e29,false,false,http://iec.ch/TC57/2013/CIM-schema-cim16#RegulatingControlModeKind.voltage,0,117.432236207162,http://iec.ch/TC57/2013/CIM-schema-cim16#UnitMultiplier.k,RegulatingControl
1,016ebf77-23d6-4b18-9419-f9fb287d53d9,87fbb934-4652-4850-ab0d-172d41ab3b19,NaN,NaN,G4,0d36e02b-95d5-bb57-beab-38db4a3c3739,false,5cb62f87-ac66-ad43-edb0-ca0b1a95d029,0,0,0,5,35,NaN,NaN,NaN,NaN,NaN,1.7,-1.7,NaN,http://iec.ch/TC57/2013/CIM-schema-cim16#SynchronousMachineOperatingMode.generator,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,http://iec.ch/TC57/2013/CIM-schema-cim16#SynchronousMachineKind.generator,NaN,NaN,NaN,SynchronousMachine,006fbe76-22d7-4a19-9518-f8fa297c52d8,false,1,NaN,T1,016ebf77-23d6-4b18-9419-f9fb287d53d9,NaN,c5f6b4a8-4444-c927-4de2-279717ecc43c,NaN,Terminal,03012d29-e556-459a-bc9a-b659f1ee3011,NaN,NaN,0,5,1,NaN,1,http://iec.ch/TC57/2013/CIM-schema-cim16#HydroEnergyConversionKind.generator,NaN,G4,HydroGeneratingUnit,NaN,G4,006fbe76-22d7-4a19-9518-f8fa297c52d8,false,false,http://iec.ch/TC57/2013/CIM-schema-cim16#RegulatingControlModeKind.voltage,0,0,http://iec.ch/TC57/2013/CIM-schema-cim16#UnitMultiplier.k,RegulatingControl
2,01abdbe0-ba9c-60b6-40f5-dcb81193017e,03a180d5-1384-33d6-2bd5-6b6b3e5eb12a,NaN,_SM_130032_WG,WGACKO1_WG,71325a48-d489-eb76-6bd9-03a451c2cc2d,true,a12a3ab0-

In [17]:
# Add powerflow results to Synchronous Machines

sv_instances = loaded_instances[loaded_instances["Model.profile"]=="http://entsoe.eu/CIM/StateVariables/4/1"]

for UUID, row in sv_instances.iterrows():

    # Get some display name
    #authority = row['Model.modelingAuthoritySet']
    authority = filename_data.query("ID == '{}'".format(UUID))["model_authority"].item()
    

    try:
        EMF_name = urlparse(authority).netloc

    except:
        print("No modelling authorityset found or invalid url -> {}, using SV UUID".format(authority))
        EMF_name = SV_UUID
        
    if EMF_name == "":
        print ("Not valid url -> " + authority + " , Using raw string")
        EMF_name = authority   
        
    # Get SV data
    SvVoltages    = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvVoltage").add_prefix(EMF_name + "_")
    SvPowerFlows  = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvPowerFlow").add_prefix(EMF_name + "_")

    # Merge SV data
    SynchronousMachines = pandas.merge(SynchronousMachines, SvVoltages,   suffixes=('', '_SvVoltage'),   how = "left", left_on = 'Terminal.TopologicalNode', right_on = EMF_name + '_SvVoltage.TopologicalNode')
    SynchronousMachines = pandas.merge(SynchronousMachines, SvPowerFlows, suffixes=('', '_SvPowerFlow'), how = "left", left_on = 'ID_Terminal', right_on = EMF_name + '_SvPowerFlow.Terminal')
    
# Add to report    
comparison_dict["report"]["SynchronousMachines"] = SynchronousMachines

Not valid url -> BALTICRSC-CE , Using raw string
Not valid url -> CGMEU , Using raw string
Not valid url -> HANS-CE , Using raw string
Not valid url -> TSCNET-CE , Using raw string


In [18]:
# Create report on Synchronous Machines powerflow results

parameters_to_compare = ["SvPowerFlow.p", "SvPowerFlow.q", "SvVoltage.v"]

for report_name in parameters_to_compare:

    # Report all SV combinations
    combinations_iterator = combinations(comparison_dict["data"].keys(), 2)

    diff_column_name_list = []
    result_column_list = []

    for name in EMF_namelist: 
        result_column_list.append(name + "_" + report_name)

    for combination in combinations_iterator:

        diff_column_name = "{} - {}_{}".format(combination[0], combination[1], report_name)
        diff_column_name_list.append(diff_column_name
                                    )
        SynchronousMachines[diff_column_name] = pandas.to_numeric(SynchronousMachines[combination[0] + "_" + report_name]) - pandas.to_numeric(SynchronousMachines[combination[1] + "_" + report_name])


    #Show results
    #display(SynchronousMachines)

    markdown_print("# " + report_name)
    #display(SynchronousMachines.T[statisitcs_columns].round(2).style.applymap(color_negative_red).highlight_min(axis=0))

    diff_comparison = SynchronousMachines.set_index("ID")[diff_column_name_list].dropna()
    print ("Number of common points -> " + str(len(diff_comparison)))
    diff = pandas.DataFrame(diff_comparison.abs().sum(), columns = ["sum diff_" + report_name]).round(2).sort_values("sum diff_" + report_name)
    display(diff)

    largest_diff = diff.tail(1).index.item()
    largest_diff_value = diff.tail(1)["sum diff_" + report_name].item()

    if float(largest_diff_value) > 0:

        markdown_print("**Max positive diff**")

        # Columns to be shown in short comparison
        show_columns = ["IdentifiedObject.name", "Type_GeneratingUnit", "RegulatingCondEq.controlEnabled", "ACDCTerminal.connected"] + result_column_list + [largest_diff]

        # Sort output
        sorted_diff = SynchronousMachines.set_index("ID")[show_columns].sort_values(largest_diff,kind= 'mergesort', ascending = False)
        
        # Show top positive diff
        display(sorted_diff.head(show_top))

        # Draw diagram
        #from_UUID = sorted_diff.index[0]
        #connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        #display(graph.show(r"{}.html".format(from_UUID)))

        markdown_print("**Max negative diff**")
        
        # Show top negative diff
        display(sorted_diff.tail(show_top))

        #from_UUID = sorted_diff.index[-1]
        #connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
        #display(graph.show(r"{}.html".format(from_UUID)))

    else:
        print("No differences")

# SvPowerFlow.p

Number of common points -> 123


,sum diff_SvPowerFlow.p
KEY,
CGMEU - HANS-CE_SvPowerFlow.p,205.61
CGMEU - TSCNET-CE_SvPowerFlow.p,229.82
HANS-CE - TSCNET-CE_SvPowerFlow.p,231.92
BALTICRSC-CE - HANS-CE_SvPowerFlow.p,285.78
BALTICRSC-CE - TSCNET-CE_SvPowerFlow.p,354.12
CGMEU - BALTICRSC-CE_SvPowerFlow.p,354.75


**Max positive diff**

KEY,IdentifiedObject.name,Type_GeneratingUnit,RegulatingCondEq.controlEnabled,ACDCTerminal.connected,BALTICRSC-CE_SvPowerFlow.p,HANS-CE_SvPowerFlow.p,TSCNET-CE_SvPowerFlow.p,CGMEU_SvPowerFlow.p,CGMEU - BALTICRSC-CE_SvPowerFlow.p
ID,,,,,,,,,
047c6158-29d0-421e-a793-4de98a2fdf2b,G3,HydroGeneratingUnit,true,true,-102.8037,-105.785,-115.986,-100,2.80370
6a16834d-90d9-47f6-85f5-ec19c6742ddc,G4,HydroGeneratingUnit,true,true,-120.8037,-124.827,-123.584,-118,2.80370
8754d38e-cec5-4328-8ef0-5b71dccf3be1,G1,HydroGeneratingUnit,true,true,-132.8037,-137.521,-131.847,-130,2.80370
988e6829-bdea-49a5-abc1-a2c7bd4ee443,AG2,ThermalGeneratingUnit,true,true,-201.8037,-210.,-201.506,-199,2.80370
a875cee8-5c66-4620-9661-0cc1fc57d4bc,G1,WindGeneratingUnit,false,true,-15.80368,-13.7521,-13.0002,-13,2.80368


...

,sum diff_SvVoltage.v
KEY,
CGMEU - TSCNET-CE_SvVoltage.v,2.15
CGMEU - HANS-CE_SvVoltage.v,2.42
HANS-CE - TSCNET-CE_SvVoltage.v,3.39
BALTICRSC-CE - HANS-CE_SvVoltage.v,5.39
CGMEU - BALTICRSC-CE_SvVoltage.v,6.01
BALTICRSC-CE - TSCNET-CE_SvVoltage.v,7.36


**Max positive diff**

KEY,IdentifiedObject.name,Type_GeneratingUnit,RegulatingCondEq.controlEnabled,ACDCTerminal.connected,BALTICRSC-CE_SvVoltage.v,HANS-CE_SvVoltage.v,TSCNET-CE_SvVoltage.v,CGMEU_SvVoltage.v,BALTICRSC-CE - TSCNET-CE_SvVoltage.v
ID,,,,,,,,,
0e69a0e1-1f21-4ffb-8472-c6c7e65949f5,G1,WindGeneratingUnit,false,true,113.2809,113.059,112.719,113.07,0.5619
5c1d36e7-4f68-42e2-85eb-f47737e707e4,G1,WindGeneratingUnit,false,true,114.1065,113.849,113.55,113.859,0.5565
3acc1de7-e4d2-446d-9d59-7eb9c75102aa,ZD6P,WindGeneratingUnit,false,true,118.2998,118.175,118.071,118.125,0.2288
fd1cc3a9-32f7-4bfe-b9ba-e03ba1c271d5,ZD6,WindGeneratingUnit,false,true,118.2998,118.175,118.071,118.125,0.2288
a875cee8-5c66-4620-9661-0cc1fc57d4bc,G1,WindGeneratingUnit,false,true,117.4892,117.43,117.37,117.411,0.1192


**Max negative diff**

KEY,IdentifiedObject.name,Type_GeneratingUnit,RegulatingCondEq.controlEnabled,ACDCTerminal.connected,BALTICRSC-CE_SvVoltage.v,HANS-CE_SvVoltage.v,TSCNET-CE_SvVoltage.v,CGMEU_SvVoltage.v,BALTICRSC-CE - TSCNET-CE_SvVoltage.v
ID,,,,,,,,,
c1a05240-9676-41dd-84b6-ae2cf08fcbc0,TG 2,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
c5409812-b41e-44ae-83f5-9403b234ea39,G1+G2,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
cd5350d0-176a-44c1-841c-63c551aef061,G1,HydroGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
e836346e-e148-4b8f-9eab-cb0ce020b27d,G2,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN
ff3853cc-0b28-46f8-b31d-27fa6bed0f82,G6,ThermalGeneratingUnit,false,false,NaN,0.,0.0,0,NaN


In [19]:
markdown_print("**Top 5 detailed data**")
display(SynchronousMachines.set_index("ID").sort_values(largest_diff,kind='mergesort', ascending = False).head(show_top))

**Top 5 detailed data**

KEY,Equipment.EquipmentContainer,Equipment.aggregate,IdentifiedObject.description,IdentifiedObject.name,RegulatingCondEq.RegulatingControl,RegulatingCondEq.controlEnabled,RotatingMachine.GeneratingUnit,RotatingMachine.p,RotatingMachine.q,RotatingMachine.ratedPowerFactor,RotatingMachine.ratedS,RotatingMachine.ratedU,SynchronousMachine.InitialReactiveCapabilityCurve,SynchronousMachine.earthing,SynchronousMachine.earthingStarPointR,SynchronousMachine.earthingStarPointX,SynchronousMachine.ikk,SynchronousMachine.maxQ,SynchronousMachine.minQ,SynchronousMachine.mu,SynchronousMachine.operatingMode,SynchronousMachine.qPercent,SynchronousMachine.r,SynchronousMachine.r0,SynchronousMachine.r2,SynchronousMachine.referencePriority,SynchronousMachine.satDirectSubtransX,SynchronousMachine.satDirectSyncX,SynchronousMachine.satDirectTransX,SynchronousMachine.shortCircuitRotorType,SynchronousMachine.type,SynchronousMachine.voltageRegulationRange,SynchronousMachine.x0,SynchronousMachine.x2,Type,ID_Terminal,ACDCTerminal.connected,ACDCTerminal.sequenceNumber,IdentifiedObject.description_Terminal,IdentifiedObject.name_Terminal,Terminal.ConductingEquipment,Terminal.ConnectivityNode,Terminal.TopologicalNode,Terminal.phases,Type_Terminal,Equipment.EquipmentContainer_GeneratingUnit,Equipment.aggregate_GeneratingUnit,GeneratingUnit.genControlSource,GeneratingUnit.initialP,GeneratingUnit.maxOperatingP,GeneratingUnit.minOperatingP,GeneratingUnit.nominalP,GeneratingUnit.normalPF,HydroGeneratingUnit.energyConversionCapability,IdentifiedObject.description_GeneratingUnit,IdentifiedObject.name_GeneratingUnit,Type_GeneratingUnit,WindGeneratingUnit.windGenUnitType,IdentifiedObject.name_RegulatingControl,RegulatingControl.Terminal,RegulatingControl.discrete,RegulatingControl.enabled,RegulatingControl.mode,RegulatingControl.targetDeadband,RegulatingControl.targetValue,RegulatingControl.targetValueUnitMultiplier,Type_RegulatingControl,BALTICRSC-CE_SvVoltage.TopologicalNode,BALTICRSC-CE_SvVoltage.angle,BALTICRSC-CE_SvVoltage.v,BALTICRSC-CE_Type,BALTICRSC-CE_SvPowerFlow.Terminal,BALTICRSC-CE_SvPowerFlow.p,BALTICRSC-CE_SvPowerFlow.q,BALTICRSC-CE_Type_SvPowerFlow,CGMEU_SvVoltage.TopologicalNode,CGMEU_SvVoltage.angle,CGMEU_SvVoltage.v,CGMEU_Type,CGMEU_SvPowerFlow.Terminal,CGMEU_SvPowerFlow.p,CGMEU_SvPowerFlow.q,CGMEU_Type_SvPowerFlow,HANS-CE_SvVoltage.TopologicalNode,HANS-CE_SvVoltage.angle,HANS-CE_SvVoltage.v,HANS-CE_Type,HANS-CE_SvPowerFlow.Terminal,HANS-CE_SvPowerFlow.p,HANS-CE_SvPowerFlow.q,HANS-CE_Type_SvPowerFlow,TSCNET-CE_SvVoltage.TopologicalNode,TSCNET-CE_SvVoltage.angle,TSCNET-CE_SvVoltage.v,TSCNET-CE_Type,TSCNET-CE_SvPowerFlow.Terminal,TSCNET-CE_SvPowerFlow.p,TSCNET-CE_SvPowerFlow.q,TSCNET-CE_Type_SvPowerFlow,CGMEU - BALTICRSC-CE_SvPowerFlow.p,CGMEU - HANS-CE_SvPowerFlow.p,CGMEU - TSCNET-CE_SvPowerFlow.p,BALTICRSC-CE - HANS-CE_SvPowerFlow.p,BALTICRSC-CE - TSCNET-CE_SvPowerFlow.p,HANS-CE - TSCNET-CE_SvPowerFlow.p,CGMEU - BALTICRSC-CE_SvPowerFlow.q,CGMEU - HANS-CE_SvPowerFlow.q,CGMEU - TSCNET-CE_SvPowerFlow.q,BALTICRSC-CE - HANS-CE_SvPowerFlow.q,BALTICRSC-CE - TSCNET-CE_SvPowerFlow.q,HANS-CE - TSCNET-CE_SvPowerFlow.q,CGMEU - BALTICRSC-CE_SvVoltage.v,CGMEU - HANS-CE_SvVoltage.v,CGMEU - TSCNET-CE_SvVoltage.v,BALTICRSC-CE - HANS-CE_SvVoltage.v,BALTICRSC-CE - TSCNET-CE_SvVoltage.v,HANS-CE - TSCNET-CE_SvVoltage.v
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0e69a0e1-1f21-4ffb-8472-c6c7e65949f5,a2043eb2-332a-4e3d-a280-815001ff5e0f,NaN,NaN,G1,a6876d86-6896-15f6-ed5d-6e27d11416e5,false,0702a381-5659-1fb6-f5e7-eb07a7b0f4d0,-4.000002,0.4363801,0,34.2,110,NaN,NaN,NaN,NaN,NaN,10.92,-10.92,NaN,http://iec.ch/TC57/2013/CIM-schema-cim16#SynchronousMachineOperatingMode.generator,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,http://iec.ch/TC57/2013/CIM-schema-cim16#SynchronousMachineKind.generator,NaN,NaN,NaN,SynchronousMachine,0f68a1e0-1e20-4efa-8573-c7c6e75848f4,true,1,NaN,T1,0e69a0e1-1f21-4ffb-8472-c6c7e65949f5,NaN

In [20]:
markdown_print("# Check that SV voltage results are not equal with base voltages")

voltages = pandas.DataFrame() # TODO - Should check that no 0 values in main island

BaseVoltages = data.type_tableview("BaseVoltage")
BaseVoltages["BaseVoltage.nominalVoltage"] = pandas.to_numeric(BaseVoltages["BaseVoltage.nominalVoltage"])
#BaseVoltages.at["11d0b6c9cf1241229fec89dbce938f82", "BaseVoltage.nominalVoltage"] = 402 # Test if logic works, by changing base voltage to mach one voltage result


for name in EMF_namelist:

    voltages[name] = comparison_dict['report'][report_name.split("_")[0]][name].dropna().value_counts()

    
#voltages.dropna()
product = pandas.merge(voltages.reset_index(), BaseVoltages, left_on = "index", right_on = "BaseVoltage.nominalVoltage", how = "inner")

if len(product)== 0:
    print("No voltages are equal to basevoltages")
    
else:
    display(product)

# Check that SV voltage results are not equal with base voltages

No voltages are equal to basevoltages


In [21]:
# Get ACLineSegments with terminals

# Generate classical data views needed to extract relevant data
ACLineSegments          = data.type_tableview("ACLineSegment").reset_index()
Terminals               = data.type_tableview("Terminal").reset_index()

# Lets first filter terminals belonging to ACLineSegments to increase performance
ACLineSegments_terminals = pandas.merge(ACLineSegments[["ID"]].rename(columns={"ID":"ACLineSegment"}),
                                        Terminals,
                                        how="inner",
                                        left_on="ACLineSegment",
                                        right_on="Terminal.ConductingEquipment").drop(columns="ACLineSegment")



# Merge terminals connected to same equipment
branches = pandas.merge(ACLineSegments_terminals, ACLineSegments_terminals, how = "inner", on = "Terminal.ConductingEquipment")

# Filter out termials refering to itself and drop out duplicate entries
branches = branches[(branches["ID_x"] != branches["ID_y"])].drop_duplicates("Terminal.ConductingEquipment") # We want only one entry per ACLineSegment, so we drop the duplicate ones

# Join views to get needed AC line data
ACLineSegments = pandas.merge(ACLineSegments, branches, how = "inner", left_on="ID", right_on = 'Terminal.ConductingEquipment')#.set_index("ID")

#display(ACLineSegments)
#print("Total number of lines {}".format(len(ACLineSegments))) #DEBUG - check that no lines have been missed

In [24]:
# Add powerflow results to ACLine Segments

sv_instances = loaded_instances[loaded_instances["Model.profile"]=="http://entsoe.eu/CIM/StateVariables/4/1"]

for UUID, row in sv_instances.iterrows():

    # Get some display name
    #authority = row['Model.modelingAuthoritySet']
    authority = filename_data.query("ID == '{}'".format(UUID))["model_authority"].item()
    

    try:
        EMF_name = urlparse(authority).netloc

    except:
        print("No modelling authorityset found or invalid url -> {}, using SV UUID".format(authority))
        EMF_name = SV_UUID
        
    if EMF_name == "":
        print ("Not valid url -> " + authority + " , Using raw string")
        EMF_name = authority  
        
    # Get SV data
    SvVoltages    = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvVoltage")
    SvPowerFlows  = data.query("INSTANCE_ID == '{}'".format(UUID)).type_tableview("SvPowerFlow")
    
    y_column_prefix = EMF_name + "_y_"
    x_column_prefix = EMF_name + "_x_"    

    # Merge SV data
    ACLineSegments = pandas.merge(ACLineSegments, SvVoltages.add_prefix(y_column_prefix),   suffixes=('', '_SvVoltage_y'),   how = "left", left_on = 'Terminal.TopologicalNode_y', right_on = y_column_prefix + 'SvVoltage.TopologicalNode')
    ACLineSegments = pandas.merge(ACLineSegments, SvPowerFlows.add_prefix(y_column_prefix), suffixes=('', '_SvPowerFlow_y'), how = "left", left_on = 'ID_y', right_on = y_column_prefix + 'SvPowerFlow.Terminal')
    ACLineSegments = pandas.merge(ACLineSegments, SvVoltages.add_prefix(x_column_prefix),   suffixes=('', '_SvVoltage_x'),   how = "left", left_on = 'Terminal.TopologicalNode_x', right_on = x_column_prefix + 'SvVoltage.TopologicalNode')
    ACLineSegments = pandas.merge(ACLineSegments, SvPowerFlows.add_prefix(x_column_prefix), suffixes=('', '_SvPowerFlow_x'), how = "left", left_on = 'ID_x', right_on = x_column_prefix + 'SvPowerFlow.Terminal')

Not valid url -> BALTICRSC-CE , Using raw string
Not valid url -> CGMEU , Using raw string
Not valid url -> HANS-CE , Using raw string
Not valid url -> TSCNET-CE , Using raw string


In [25]:
# Recalculate flows on lines
from numpy import sin, cos, deg2rad, triu, pi


input_data = ACLineSegments

for EMF_name in EMF_namelist:

    # Define input columns
    r_name = "ACLineSegment.r"
    x_name = "ACLineSegment.x"
    C_name = "ACLineSegment.bch"
    L_name = "ACLineSegment.gch"
    

    u_1_name = '{}_x_SvVoltage.v'.format(EMF_name)
    u_2_name = '{}_y_SvVoltage.v'.format(EMF_name)

    angle_1_name = '{}_x_SvVoltage.angle'.format(EMF_name)
    angle_2_name = '{}_y_SvVoltage.angle'.format(EMF_name)

    # Define output columns

    p_1_name = '{}_x_SvPowerFlow.p_calc'.format(EMF_name)
    p_2_name = '{}_y_SvPowerFlow.p_calc'.format(EMF_name)
    d_p_name = '{}_ACLineSegment.delta_p'.format(EMF_name)

    q_1_name = '{}_x_SvPowerFlow.q_calc'.format(EMF_name)
    q_2_name = '{}_y_SvPowerFlow.q_calc'.format(EMF_name)
    d_q_name = '{}_ACLineSegment.delta_q'.format(EMF_name)



    # Set input variable types
    r = input_data[r_name].astype("float")
    x = input_data[x_name].astype("float")
    C = input_data[C_name].astype("float")

    u_1 = input_data[u_1_name].astype("float")
    u_2 = input_data[u_2_name].astype("float")

    angle_1 = deg2rad(input_data[angle_1_name].astype("float"))
    angle_2 = deg2rad(input_data[angle_2_name].astype("float"))

    # ACLineSegment formulas
    
    #Inductive reactance    
    f=50 #hZ

    #XL = 2 * pi * f * L 
    xC = 1 / (2 * pi * f * C)

    # Calculate admittance
    Y = 1/(r + (x - xC) * 1j)
    #Z = r + x * 1j
    


    # Calcualte complex voltages
    V_1 = u_1 * cos(angle_1) + u_1 * sin(angle_1) * 1j
    V_2 = u_2 * cos(angle_2) + u_2 * sin(angle_2) * 1j

    # Calcualte flows at nodes
    S_1 = V_1 * (Y * V_1 - Y * V_2)
    S_2 = V_2 * (Y * V_2 - Y * V_1)

    p_1 = S_1.real
    p_2 = S_2.real
    d_p = abs(p_1 + p_2)

    q_1 = S_1.imag
    q_2 = S_2.imag
    d_q = abs(q_1 + q_2)

    #display(d_p)

    # Write results
    input_data[p_1_name] = p_1
    input_data[p_2_name] = p_2
    input_data[d_p_name] = d_p

    input_data[q_1_name] = q_1
    input_data[q_2_name] = q_2
    input_data[d_q_name] = d_q


    #S = I * V
    #I = V / R

# Add to report    
comparison_dict["report"]["ACLineSegments"] = ACLineSegments

In [ ]:
#ACLineSegments # DEBUG

In [26]:
# Create report on ACLineSegments powerflow results

parameters_to_compare = ["ACLineSegment.delta_p", "ACLineSegment.delta_q"]

report_data = pandas.DataFrame(ACLineSegments[(ACLineSegments["ACDCTerminal.connected_x"] == "true") & (ACLineSegments["ACDCTerminal.connected_y"] == "true")])

show_graphs = False


for report_name in parameters_to_compare:

    # Report all SV combinations
    combinations_iterator = combinations(comparison_dict["data"].keys(), 2)

    diff_column_name_list = []
    result_column_list = []

    for name in EMF_namelist: 
        result_column_list.append(name + "_" + report_name)

    for combination in combinations_iterator:

        diff_column_name = "{} - {}_{}".format(combination[0], combination[1], report_name)
        diff_column_name_list.append(diff_column_name
                                    )
        report_data[diff_column_name] = pandas.to_numeric(report_data[combination[0] + "_" + report_name]) - pandas.to_numeric(report_data[combination[1] + "_" + report_name])



    markdown_print("# " + report_name)


    diff_comparison = report_data.set_index("ID")[diff_column_name_list].dropna()
    print ("Number of common points -> " + str(len(diff_comparison)))
    diff = pandas.DataFrame(diff_comparison.abs().sum(), columns = ["sum diff_" + report_name]).round(2).sort_values("sum diff_" + report_name)
    display(diff)

    largest_diff = diff.tail(1).index.item()
    largest_diff_value = diff.tail(1)["sum diff_" + report_name].item()

    if float(largest_diff_value) > 0:

        markdown_print("**Max positive diff**")

        # Columns to be shown in short comparison
        show_columns = ["IdentifiedObject.name"] + result_column_list + [largest_diff]

        # Sort output
        sorted_diff = report_data.set_index("ID")[show_columns].sort_values(largest_diff,kind= 'mergesort', ascending = False)
        
        # Show top positive diff
        display(sorted_diff.head(show_top))

        
        if show_graphs == True:
            # Draw diagram
            from_UUID = sorted_diff.index[0]
            connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
            display(graph.show(r"{}.html".format(from_UUID)))

        markdown_print("**Max negative diff**")
        
        # Show top negative diff
        display(sorted_diff.tail(show_top))

        if show_graphs == True:
            
            from_UUID = sorted_diff.index[-1]
            connections, identified_objects, graph = get_relations_from(data, from_UUID, notebook = True)
            display(graph.show(r"{}.html".format(from_UUID)))

    else:
        print("No differences")

# ACLineSegment.delta_p

Number of common points -> 581


,sum diff_ACLineSegment.delta_p
KEY,
HANS-CE - TSCNET-CE_ACLineSegment.delta_p,0.00
CGMEU - BALTICRSC-CE_ACLineSegment.delta_p,106.07
BALTICRSC-CE - HANS-CE_ACLineSegment.delta_p,430.48
BALTICRSC-CE - TSCNET-CE_ACLineSegment.delta_p,430.48
CGMEU - HANS-CE_ACLineSegment.delta_p,441.39
CGMEU - TSCNET-CE_ACLineSegment.delta_p,441.39


**Max positive diff**

KEY,IdentifiedObject.name,BALTICRSC-CE_ACLineSegment.delta_p,HANS-CE_ACLineSegment.delta_p,TSCNET-CE_ACLineSegment.delta_p,CGMEU_ACLineSegment.delta_p,CGMEU - TSCNET-CE_ACLineSegment.delta_p
ID,,,,,,
fa5c533a-beb4-f22f-0bbd-764c2e1b9d8f,WTSTAN1_WBLUKA1_1_400,27.409897,5.304136,5.304136,31.847993,26.543857
965de4be-fa2e-4079-b14d-830f4ad49df3,Melina_HE Senj,33.254682,9.578389,9.578389,35.619070,26.040681
ef4df330-d5f1-c8db-14d3-95aac4da06cc,WTETUZ2_WGRADC2_1_220,7.872475,3.457080,3.457080,8.167113,4.710033
60790718-d500-45d0-a418-4f1ff9cf6e69,Ernestinovo_Ugljevik,4.962659,0.714306,0.714306,5.189604,4.475299
a76139a1-494d-ab92-8a99-6738b0ceb8af,WTUZLA1_WSAR101_1_400,4.327113,3.538707,3.538707,7.456663,3.917955


...

,sum diff_ACLineSegment.delta_q
KEY,
HANS-CE - TSCNET-CE_ACLineSegment.delta_q,0.00
CGMEU - BALTICRSC-CE_ACLineSegment.delta_q,108.58
BALTICRSC-CE - HANS-CE_ACLineSegment.delta_q,396.55
BALTICRSC-CE - TSCNET-CE_ACLineSegment.delta_q,396.55
CGMEU - HANS-CE_ACLineSegment.delta_q,461.97
CGMEU - TSCNET-CE_ACLineSegment.delta_q,461.97


**Max positive diff**

KEY,IdentifiedObject.name,BALTICRSC-CE_ACLineSegment.delta_q,HANS-CE_ACLineSegment.delta_q,TSCNET-CE_ACLineSegment.delta_q,CGMEU_ACLineSegment.delta_q,CGMEU - TSCNET-CE_ACLineSegment.delta_q
ID,,,,,,
34d972ad-cd20-4408-bd75-9e85892298d1,BER-POD400,141.890946,76.639197,76.639197,169.204486,92.565289
bd2d6659-d6c8-4acc-8a73-e4908af3f2a5,DIV-RED,53.171537,5.907695,5.907695,57.841279,51.933584
c4a397f5-8fe1-4ca8-a048-46714935cc12,BER-DIV,94.207526,44.338504,44.338504,94.171840,49.833335
fc472235-89e8-8c8e-4b13-05724b154e01,XMO_KO11_WMOST41_1_400,28.648098,13.805346,13.805346,36.435004,22.629657
f24b8ec6-8754-4634-807f-f021ebc30692,Zerjavinec_Heviz,26.199715,7.127341,7.127341,26.865402,19.738061


**Max negative diff**

KEY,IdentifiedObject.name,BALTICRSC-CE_ACLineSegment.delta_q,HANS-CE_ACLineSegment.delta_q,TSCNET-CE_ACLineSegment.delta_q,CGMEU_ACLineSegment.delta_q,CGMEU - TSCNET-CE_ACLineSegment.delta_q
ID,,,,,,
fa5c533a-beb4-f22f-0bbd-764c2e1b9d8f,WTSTAN1_WBLUKA1_1_400,22.159295,34.863937,34.863937,15.102143,-19.761794
15af5013-aac1-4f30-a610-00b12ad08fce,LIT-TET,NaN,NaN,NaN,NaN,NaN
5ffa8b04-112f-4984-bb54-bc9d3fb09ec3,BER-LIT,NaN,NaN,NaN,NaN,NaN
c09d4fce-7c6a-47f9-bd69-4b4bbff7cc5b,HEKRS-K.DES,NaN,NaN,NaN,NaN,NaN
dfed4977-41fb-4403-882f-80d0410ae524,KOP-ENPDEK,NaN,NaN,NaN,NaN,NaN


In [27]:
#ACLineSegments[(ACLineSegments["ACDCTerminal.connected_x"] == "true") & (ACLineSegments["ACDCTerminal.connected_y"] == "true")]# DEBUG

In [28]:
#for column in ACLineSegments.columns: print("'" + column + "'") # DEBUG

In [29]:
# Sanity check against Neplan export (neplan has p and q on all ends)
#ACLineSegments[['eles.si_x_SvPowerFlow.p', 'eles.si_x_SvPowerFlow.p_calc']]
#display(abs(ACLineSegments['eles.si_x_SvPowerFlow.p'].astype("float") + ACLineSegments['eles.si_y_SvPowerFlow.p'].astype("float")).dropna().sort_values()) # Original Neplan
#display(ACLineSegments['eles.si_ACLineSegment.delta_p'].dropna().sort_values()) # Calculated Neplan

In [32]:
# Sanity check against Neplan export (neplan has p and q on all ends)
#EMF_tool = "862de2ce-d190-4245-af80-1c428d98910a"
EMF_tool = "BALTICRSC-CE"

pandas.set_option("display.max_rows", 30)
ACLineSegments["diff"] = ACLineSegments['{}_x_SvPowerFlow.p'.format(EMF_tool)].astype("float") - ACLineSegments['{}_x_SvPowerFlow.p_calc'.format(EMF_tool)].astype("float")
diff = ACLineSegments[(ACLineSegments["ACDCTerminal.connected_x"] == "true") & (ACLineSegments["ACDCTerminal.connected_y"] == "true")][['ID', 'IdentifiedObject.name', 
                                                                                                                                '{}_x_SvPowerFlow.p'.format(EMF_tool),
                                                                                                                                '{}_x_SvPowerFlow.p_calc'.format(EMF_tool),
                                                                                                                                'diff',
                                                                                                                                "ACLineSegment.r", "ACLineSegment.x", "ACLineSegment.bch", "ACLineSegment.gch",
                                                                                                                                '{}_x_SvVoltage.angle'.format(EMF_tool),
                                                                                                                                '{}_x_SvVoltage.v'.format(EMF_tool),
                                                                                                                                 '{}_y_SvVoltage.angle'.format(EMF_tool),
                                                                                                                                 '{}_y_SvVoltage.v'.format(EMF_tool)]
                                                                                                                                      ].dropna().sort_values("diff")

diff.to_csv("calculated_PF_diff_{}.csv".format(EMF_tool))

display(diff)

KEY,ID,IdentifiedObject.name,BALTICRSC-CE_x_SvPowerFlow.p,BALTICRSC-CE_x_SvPowerFlow.p_calc,diff,ACLineSegment.r,ACLineSegment.x,ACLineSegment.bch,ACLineSegment.gch,BALTICRSC-CE_x_SvVoltage.angle,BALTICRSC-CE_x_SvVoltage.v,BALTICRSC-CE_y_SvVoltage.angle,BALTICRSC-CE_y_SvVoltage.v
448,bd2d6659-d6c8-4acc-8a73-e4908af3f2a5,DIV-RED,-866.9886,813.195449,-1680.184049,0.8579848,11.51875,0.0001514206,0,-21.68962,416.4991,-18.31549,413.8797
368,9dfe3de4-74b0-45d6-a339-ad8720aa5cef,TS Konjsko_Mostar 4,-38.67484,521.334681,-560.009521,1.752,17.712,0.000196059,0,-2.977645,427.986,-2.672604,421.6374
467,c4a397f5-8fe1-4ca8-a048-46714935cc12,BER-DIV,572.2079,1084.296775,-512.088875,2.361394,24.83272,0.0002699391,0,-6.109088,406.7133,-11.06421,408.9941
533,de5c1d9a-f688-49fd-82eb-b0ed02d3f5fb,MAR-KAI1,-305.3355,78.564798,-383.900298,0.7109557,7.423686,8.890493E-05,0,-2.012306,407.9077,-1.233986,408.5402
347,965de4be-fa2e-4079-b14d-830f4ad49df3,Melina_HE Senj,143.6154,435.947139,-292.331739,4.42,23.69,0.00014749,0,-4.782646,239.2748,-8.281842,239.0613
14,0571d504-dbf2-42ca-a4e9-42a9d025338c,Tumbri_NE Krsko,-194.6909,53.376167,-248.067067,0.716,7.574,8.7935E-05,0,-5.357831,410.2619,-4.8428,409.6838
202,5d927f6b-7fc1-412a-919d-793dd922bf23,Tumbri_NE Krsko,-194.6648,53.383795,-248.048595,0.716,7.574,8.7935E-05,0,-5.357831,410.2619,-4.842819,409.6805
516,d798d7a2-0e68-42da-b24c-72f9b2814b7a,NEK-ZAG2,-194.837,20.982736,-215.819736,0.483,5.2325,5.651263E-05,0,-4.842819,409.6805,-4.484049,409.1495
586,f24b8ec6-8754-4634-807f-f021ebc30692,Zerjavinec_Heviz,320.5512,535.073435,-214.522235,2.555,26.58,0.00029479,0,-2.12868,413.9628,-4.981395,411.3537
400,ac64a807-e24a-4e80-863d-0d2ee00dbe03,HE Zakucac_Mostar 4,27.41092,227.665729,-200.254809,3.79,19.79,0.0001297,0,-1.532238,251.4115,-1.658643,242.5262


In [31]:
display(HTML("<style>.container { width:90% !important; }</style>")) # Change report layout width

In [ ]:
# Export to excel results

#export_path  = r"C:\IOPs\IOP150519\RSC_MERGE\SV_comparison_150519.xlsx" 
#excel_writer = pandas.ExcelWriter(export_path)

#for report in comparison_dict["report"].keys():
#    comparison_dict["report"][report].to_excel(excel_writer, sheet_name = report)

#excel_writer.save()

In [ ]:
"EnergyConsumer", "ConfromLoad", "NonConfromLoad"

data.type_tableview("EnergyConsumer")


In [ ]:
pandas.merge(data.query("ID == '01e7ddda-3405-49e1-98f1-85dd390cae2e'"), loaded_instances, left_on = "INSTANCE_ID", right_index = True)[[u'ID', u'KEY', u'VALUE', u'INSTANCE_ID', u'Model.profile', u'Model.createdBy', u'Model.description', u'Model.modelingAuthoritySet']]

In [ ]:
#loaded_instances